In [20]:
import numpy as np
import tensorflow as tf
from sympy import solve
from sympy import Number
from sympy.abc import a, b, c, d, e, f, g, I
from sympy.core.add import Add
from sympy.core.symbol import Symbol
from typing import List, Dict
from dataclasses import dataclass, field
from functools import reduce
from IPython.display import display, Image

In [2]:
for expression in [0.5*(a - b) - I, 0.1*(a - d) - I]:
    print(solve(expression, I))
    print(type(solve(expression, I)[0]))

[0.5*a - 0.5*b]
<class 'sympy.core.add.Add'>
[0.1*a - 0.1*d]
<class 'sympy.core.add.Add'>


In [4]:
@dataclass
class CircuitNode:
    label: str
    resistor_exprs: List[Add]
    current_symbol: Symbol
    node_expr: Add = field(default_factory=Add)

    def __post_init__(self) -> None:
        has_c_symbol: Dict[int, bool] = {
            idx: le.has(self.current_symbol) for idx, le in enumerate(self.resistor_exprs)
        }
        if not all(has_c_symbol.values()):
            missing_symbol: Dict[int, bool] = {
                idx: test_val for idx, test_val in has_c_symbol.items() if not test_val
            }
            msg: str = f"""
            All resistor expressions must contain the current symbol!
            The following do not:
            {missing_symbol}
            """
            raise ValueError(msg)
            
        current_expressions: List[Add] = CircuitNode.solve_for_current(self.resistor_exprs)
        self.node_expr: Add = CircuitNode.combine_resistor_exprs(current_expressions)

    @staticmethod
    def solve_for_current(resistor_exprs: List[Add]) -> List[Add]:
        out: List[Add] = list(map(lambda le: solve(le, I)[0], resistor_exprs))
        return out
    
    @staticmethod
    def combine_resistor_exprs(resistor_exprs: List[Add]) -> Add:
        out: Add = reduce(lambda first, second: first + second, resistor_exprs)
        return out

x5: CircuitNode = CircuitNode("x5", [0.5*(e - b) - I, 0.1*(e - d) - I, .5*(e - f) - I, .1*(e - 9) - I], I)
    
x5

CircuitNode(label='x5', resistor_exprs=[-I - 0.5*b + 0.5*e, -I - 0.1*d + 0.1*e, -I + 0.5*e - 0.5*f, -I + 0.1*e - 0.9], current_symbol=I, node_expr=-0.5*b - 0.1*d + 1.2*e - 0.5*f - 0.9)

In [5]:
CircuitNode.solve_for_current(x1.resistor_exprs)

[0.5*a - 0.5*b, 0.1*a - 0.1*d]

In [11]:
x5.node_expr

-0.5*b - 0.1*d + 1.2*e - 0.5*f - 0.9

In [21]:
x5.node_expr.atoms(Number)

{-0.900000000000000, -0.500000000000000, -0.100000000000000, 1.20000000000000}